### LogRegress history based on all trials
- Miller et al. 2021

In [ ]:
import numpy as np
import pandas as pd
from banditpy.analyses import HistoryBasedLogisticModel
import mab_subjects

exps = mab_subjects.struc.allsess + mab_subjects.unstruc.allsess

coef_df = []

for i, exp in enumerate(exps):
    mab = exp.mab.keep_by_trials(min_trials=100)
    print(exp.sub_name)
    model = HistoryBasedLogisticModel(mab, n_past=10)
    model.fit()

    df = model.get_coef_df(form="long")
    df["name"] = exp.sub_name
    df["grp"] = "struc" if mab.is_structured else "unstruc"

    coef_df.append(df)

coef_df = pd.concat(coef_df, ignore_index=True)
mab_subjects.GroupData().save(coef_df, "history_coef_alltrials_anirudh")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw

fig = plotting.Fig(1, 3, size=(11, 3), num=1)

grpdata = mab_subjects.GroupData()
coef_df = grpdata.history_coef_alltrials_anirudh

coef_names = ["reward seeking", "choice preservation", "effect of outcome"]
for c, coef_name in enumerate(coef_names):

    df = coef_df[coef_df["coef_name"] == coef_name]

    ax = fig.subplot(fig.gs[c])
    ax.axhline(0, color="gray", lw=0.8, zorder=0)

    plot_kw = dict(
        data=df,
        x="past_id",
        y="coef",
        hue="grp",
        hue_order=["unstruc", "struc"],
        ax=ax,
    )
    sns.stripplot(
        # palette=["#5040BF", "#AFBF40"],
        palette=["gray", "gray"],
        size=3,
        alpha=0.5,
        dodge=True,
        **plot_kw,
    )
    sns.pointplot(
        dodge=0.4,
        palette="husl",
        linestyle="none",
        markersize=4,
        errorbar="se",
        err_kws={"linewidth": 0.95},
        **plot_kw,
    )

    orders = np.arange(1, 11)
    pairs = [((_, "unstruc"), (_, "struc")) for _ in orders]
    annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
    annotator.configure(test="Kruskal", **stat_kw, color="k", verbose=True)
    annotator.apply_and_annotate()
    annotator.reset_configuration()

    ax.set_title(coef_name)
    ax.legend("")

### LogRegress histroy based on first 100 trials


In [ ]:
import numpy as np
import pandas as pd
from banditpy.analyses import HistoryBasedLogisticModel
import mab_subjects

exps = mab_subjects.struc.allsess + mab_subjects.unstruc.allsess

coef_df = []

for i, exp in enumerate(exps):
    mab = exp.mab.keep_by_trials(min_trials=100, clip_max=100)
    print(exp.sub_name)
    model = HistoryBasedLogisticModel(mab, n_past=10)
    model.fit()

    df = model.get_coef_df(form="long")
    df["name"] = exp.sub_name
    df["grp"] = "struc" if mab.is_structured else "unstruc"

    coef_df.append(df)

coef_df = pd.concat(coef_df, ignore_index=True)
mab_subjects.GroupData().save(coef_df, "history_coef_100trials_anirudh")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw

fig = plotting.Fig(8, 4, size=(8.5, 11), num=1)

grpdata = mab_subjects.GroupData()
coef_df = grpdata.history_coef_100trials_anirudh
# naive_bool = coef_df["name"].str.contains("Exp1")
# coef_df = coef_df[naive_bool]

coef_names = ["reward seeking", "choice preservation", "effect of outcome"]
for c, coef_name in enumerate(coef_names):

    df = coef_df[coef_df["coef_name"] == coef_name]

    ax = fig.subplot(fig.gs[c])
    ax.axhline(0, color="gray", lw=0.8, zorder=0)

    plot_kw = dict(
        data=df,
        x="past_id",
        y="coef",
        hue="grp",
        hue_order=["unstruc", "struc"],
        ax=ax,
    )
    sns.stripplot(
        # palette=["#5040BF", "#AFBF40"],
        palette=["gray", "gray"],
        size=2.2,
        alpha=0.5,
        dodge=True,
        **plot_kw,
    )
    sns.pointplot(
        dodge=0.4,
        palette="husl",
        linestyle="-",
        linewidth=1,
        markersize=3.2,
        errorbar="se",
        err_kws={"linewidth": 0.95},
        **plot_kw,
    )

    orders = np.arange(1, 11)
    pairs = [((_, "unstruc"), (_, "struc")) for _ in orders]
    annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
    annotator.configure(test="Kruskal", **stat_kw, color="k", verbose=False)
    annotator.apply_and_annotate()
    annotator.reset_configuration()

    ax.set_title(coef_name + "\n(All Experience)")
    ax.get_legend().remove()

### Decay constants estimation using curve fitting
- Code for both All experiences and first experiences

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statannotations.Annotator import Annotator
from statplot_utils import stat_kw
from scipy.optimize import curve_fit
import pandas as pd


def exp_decay(x, a, lambd, const):
    return a * np.exp(-lambd * x) + const


grpdata = mab_subjects.GroupData()
coef_df = grpdata.history_coef_100trials_anirudh
# naive_bool = coef_df["name"].str.contains("Exp1")
# coef_df = coef_df[naive_bool]

coef_names = ["reward seeking", "choice preservation"]

lambda_df = []
for c, coef_name in enumerate(coef_names):

    df = coef_df[coef_df["coef_name"] == coef_name]
    sub_names = df["name"].unique()

    for name in sub_names:
        sub_df = df[df["name"] == name]
        x = sub_df["past_id"].to_numpy()
        y = sub_df["coef"].to_numpy()
        grp = sub_df["grp"].unique()

        # Fit the curve
        popt, pcov = curve_fit(exp_decay, x, y, maxfev=10000)
        A_fit, lambda_fit, const = popt

        # Predicted values
        y_fit = exp_decay(x, A_fit, lambda_fit, const)

        sub_lambda_df = pd.DataFrame(
            {"lambda": lambda_fit, "coef_name": coef_name, "name": name, "grp": grp}
        )

        lambda_df.append(sub_lambda_df)


lambda_df = pd.concat(lambda_df).reset_index(drop=True)


fig = plotting.Fig(8, 8, size=(8.5, 11), num=1)

for c, coef_name in enumerate(coef_names):

    df = lambda_df[lambda_df["coef_name"] == coef_name]

    ax = fig.subplot(fig.gs[c + 3])
    plot_kw = dict(data=df, x="grp", y="lambda", ax=ax)
    sns.stripplot(data=df, x="grp", y="lambda", palette=["k", "k"], size=3)
    sns.barplot(
        data=df,
        x="grp",
        y="lambda",
        hue="grp",
        palette=["#00897B", "#F06292"],
        alpha=0.5,
        errorbar="se",
    )

    orders = ["struc", "unstruc"]
    pairs = [("struc", "unstruc")]
    annotator = Annotator(pairs=pairs, order=orders, **plot_kw)
    annotator.configure(test="Kruskal", **stat_kw, color="k", verbose=False)
    annotator.apply_and_annotate()
    annotator.reset_configuration()

    ax.set_title(f"{coef_name}")

fig.fig.suptitle("Decay constants (All experience)", fontsize=8)
# fig.fig.suptitle("Decay constants (First experience)", fontsize=8)

# ax.set_title(coef_name + "\n(All Experience)")
# ax.get_legend().remove()